<a href="https://colab.research.google.com/github/37stu37/FFE/blob/master/FFE_network_one_file_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
%%time 
%%capture
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 

CPU times: user 189 ms, sys: 54.9 ms, total: 244 ms
Wall time: 34.4 s


In [100]:
# Load the Drive helper and mount
from google.colab import drive
%tensorflow_version 2.x
# This will prompt for authorization.
drive.mount('/content/drive')
# pip freeze --local > /content/gdrive/My\ Drive/Colab Notebooks/pip_installed.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [101]:
%%time
import datetime
import glob
import math
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
import networkx as nx
from shapely.geometry import Point
import imageio

pd.options.mode.chained_assignment = None  # default='warn'

CPU times: user 86 µs, sys: 8 µs, total: 94 µs
Wall time: 95.6 µs


Set up the path  to data and output

In [0]:
path = '/content/drive/My Drive/05_Sync/FFE/Mesa'
path_output = '/content/drive/My Drive/05_Sync/FFE/Mesa/output'

# !ls "/content/drive/My Drive/05_Sync/FFE/Mesa"

Create the functions to be used by the algorithm

In [0]:
def load_data(file_name, minx, miny, maxx, maxy):
    # crop data
    bbox = box(minx, miny, maxx, maxy)
    # building point dataset
    gdf_buildings = gpd.read_file(os.path.join(path, file_name), bbox=bbox)
    # gdf_buildings.IgnProb_bl = 0.02
    # xmin,ymin,xmax,ymax = gdf_buildings.total_bounds
    return gdf_buildings


def wind_scenario():
    wind_data = pd.read_csv(os.path.join(path, 'GD_wind.csv'))
    i = np.random.randint(0, wind_data.shape[0])
    w = wind_data.iloc[i, 2]
    d = wind_data.iloc[i, 1]
    b = wind_data.iloc[i, 3]
    return w, d, b


def eudistance(v1, v2):
    return np.linalg.norm(v1 - v2)


def calculate_azimuth(x1, y1, x2, y2):
    azimuth = math.degrees(math.atan2((x2 - x1), (y2 - y1)))
    return 360 + azimuth


def plot(df, column_df):
    fig, ax = plt.subplots(1, 1)
    df.plot(column=column_df, ax=ax, legend=True)
    plt.show()


def build_edge_list(geodataframe, maximum_distance, polygon_file):
    # create arrays for different id combination
    n = np.arange(0, len(geodataframe))
    target = [n] * len(geodataframe)
    target = np.hstack(target)
    source = np.repeat(n, len(geodataframe))
    # put arrays in dataframe
    df = pd.DataFrame()
    df['source_id'] = source
    df['target_id'] = target
    # merge source attributes with source index
    geo_df = geodataframe.copy()
    geo_df['id'] = geo_df.index
    # create source / target gdf from gdf.columns of interest
    geo_df = geo_df[['id', 'TARGET_FID', 'X', 'Y', 'geometry', 'IgnProb_bl']]
    geo_df_TRG = geo_df.copy()
    geo_df_TRG.columns = ['target_' + str(col) for col in geo_df_TRG.columns]
    geo_df_SRC = geo_df.copy()
    geo_df_SRC.columns = ['source_' + str(col) for col in geo_df_SRC.columns]
    # merge data
    merged_data = pd.merge(df, geo_df_SRC, left_on='source_id', right_on='source_id', how='outer')
    merged_data = pd.merge(merged_data, geo_df_TRG, left_on='target_id', right_on='target_id', how='outer')
    merged_data.rename(columns={'source_id': 'source', 'target_id': 'target'}, inplace=True)
    # calculate distance for each source / target pair
    # create a df from polygon shape to get accurate distance
    # print(list(polygon_file))
    polygon = polygon_file[['TARGET_FID', 'geometry']]
    # print(list(polygon))
    source_poly = merged_data[['source_TARGET_FID']]
    target_poly = merged_data[['target_TARGET_FID']]
    # print(list(source_poly))
    src_poly = pd.merge(source_poly, polygon, left_on='source_TARGET_FID', right_on='TARGET_FID', how='left')
    trg_poly = pd.merge(target_poly, polygon, left_on='target_TARGET_FID', right_on='TARGET_FID', how='left')
    src_poly_gdf = gpd.GeoDataFrame(src_poly, geometry='geometry')
    trg_poly_gdf = gpd.GeoDataFrame(trg_poly, geometry='geometry')
    distance_series = src_poly_gdf.distance(trg_poly_gdf)
    # print(distance_series)

    # insert distance in merged data column
    merged_data['v1'] = merged_data.source_X - merged_data.target_X
    merged_data['v2'] = merged_data.source_Y - merged_data.target_Y
    # merged_data['euc_distance'] = np.hypot(merged_data.v1, merged_data.v2)
    merged_data['euc_distance'] = distance_series
    # remove when distance "illegal"
    valid_distance = merged_data['euc_distance'] < maximum_distance
    not_same_node = merged_data['euc_distance'] != 0
    data = merged_data[valid_distance & not_same_node]
    # calculate azimuth
    data['azimuth'] = np.degrees(np.arctan2(merged_data['v2'], merged_data['v1']))
    data['bearing'] = (data.azimuth + 360) % 360
    return data


def create_network(edge_list_dataframe):
    graph = nx.from_pandas_edgelist(edge_list_dataframe, edge_attr=True)
    # options = {'node_color': 'red', 'node_size': 50, 'width': 1, 'alpha': 0.4,
    #            'with_labels': False, 'font_weight': 'bold'}
    # nx.draw_kamada_kawai(graph, **options)
    # plt.show()
    return graph

In [0]:
def set_initial_fire_to(df):
    """Fine = 0, Fire = 1, Burned = 2"""
    df['RNG'] = np.random.uniform(0, 1, size=len(df))  # add for random suppression per building, df.shape[0])
    onFire = df['source_IgnProb_bl'] > df['RNG']
    ignitions = df[onFire]
    # source nodes ignited
    sources_on_fire = list(ignitions.source)
    sources_on_fire = list(dict.fromkeys(sources_on_fire))
    return sources_on_fire


def set_fire_to(df, existing_fires):
    are_set_on_fire = (df['source'].isin(existing_fires))
    spark = df[are_set_on_fire]
    # source nodes ignited
    sources_on_fire = list(spark.source)
    sources_on_fire = list(dict.fromkeys(sources_on_fire))
    return sources_on_fire


def fire_spreading(list_fires, list_burn, wind_speed, wind_bearing, suppression_threshold, step_value, data):
    # check the fire potential targets
    # print("fire list before spreading : {}, length : {}".format(fire_list, len(fire_list)))
    are_potential_targets = (data['source'].isin(list_fires))
    are_not_already_burned = (~data['target'].isin(list_burn))
    df = data[are_potential_targets & are_not_already_burned]
    if df.empty:
        # print("no fires")
        list_burn.extend(list(list_fires))
        list_burn = list(dict.fromkeys(list_burn))
        return [], list_burn  # to break the step loop
    # set up additional CONDITIONS for fire spreading

    # neighbors selection from buffer
    df['buffer_geometry'] = gdf.geometry.buffer(gdf['d_long'] + wind_speed)

    are_neighbors = df['euc_distance'] < wind_speed
    # print("neighbors affected ? {}".format(list(dict.fromkeys(list(are_neighbors)))))
    df = df[are_neighbors]
    # wind direction
    wind_bearing_max = wind_bearing + 45
    wind_bearing_min = wind_bearing - 45
    if wind_bearing == 360:
        wind_bearing_max = 45
    if wind_bearing <= 0:  # should not be necessary
        wind_bearing_min = 0
    if wind_bearing == 999:
        wind_bearing_max = 999
        wind_bearing_min = 0
    are_under_the_wind = (df['bearing'] < wind_bearing_max) & (df['bearing'] > wind_bearing_min)
    # print("targets under the wind ? {}".format(list(dict.fromkeys(list(are_under_the_wind)))))
    df = df[are_under_the_wind]
    # suppression
    df['random'] = np.random.uniform(0, 1, size=len(df))
    are_not_suppressed = df['random'] > suppression_threshold
    # print("fire suppressed ? {}".format(list(dict.fromkeys(list(are_not_suppressed)))))
    df = df[are_not_suppressed]

    # spread fire based on condition
    fire_df = df
    # fire_df = df[are_neighbors & are_under_the_wind & are_not_suppressed]  # issues with "are_under_the_wind
    # print(len(fire_df.head(5)))
    # print(len(fire_df))
    list_burn.extend(list(list_fires))
    fire_df['step'] = step_value
    fire_df.to_csv(os.path.join(path_output, "step{}_fire.csv".format(step_value)))
    list_fires = list(dict.fromkeys(list(fire_df.target)))
    list_burn.extend(list(fire_df.target))
    list_burn = list(dict.fromkeys(list_burn))
    return list_fires, list_burn


def log_files_concatenate(prefix, scenario_count):
    list_df = []
    files = glob.glob(os.path.join(path_output, prefix))
    if files:
        for file in files:
            # print(file)
            df = pd.read_csv(os.path.join(path_output, file))
            list_df.append(df)
            os.remove(file)
        data = pd.concat(list_df)
        data['scenario'] = scenario_count
        data.to_csv(os.path.join(path_output, "fire_scenario_{}.csv".format(scenario_count)))
    else:
        print("no files to concatenate")


def clean_up_file(prefix, path_path=path_output):
    files = glob.glob(os.path.join(path_path, prefix))
    for file in files:
        # print(file)
        os.remove(file)


def postprocessing(scenarios_recorded, burned_asset, edge_list, gdf_polygons):
    list_of_tuples = list(zip(scenarios_recorded, burned_asset))
    df = pd.DataFrame(list_of_tuples, columns=['scenarios', 'burned_asset_index'])
    # df['count'] = df['burned_asset_index'].value_counts().values
    df['count'] = df.groupby('burned_asset_index')['burned_asset_index'].transform('count')
    print(df.describe())
    df = df[['burned_asset_index', 'count']].drop_duplicates()
    edge = edge_list[
        ['source', 'source_TARGET_FID', 'source_X', 'source_Y', 'source_geometry']]
    df_id = pd.merge(df, edge, left_on='burned_asset_index', right_on='source', how='left')
    # print(list(df_id))
    df_count = pd.merge(gdf_polygons, df_id, left_on='TARGET_FID', right_on='source_TARGET_FID', how='outer')
    df_count = df_count.drop_duplicates()
    dataframe = pd.DataFrame(df_count.drop(columns=['geometry', 'source_geometry']))
    dataframe = dataframe.dropna()
    fig, ax = plt.subplots(1, 1)
    df_count.plot(column='count', cmap='RdYlBu_r', ax=ax, legend=True)
    ax.title.set_text("Burned buildings after {} scenarios".format(max(scenarios_recorded)))
    plt.savefig(os.path.join(path_output, "results_{}.png".format(number_of_scenarios)))
    # plt.show()
    plt.close(fig)
    df_count = df_count.drop(columns=['source', 'source_TARGET_FID', 'source_X', 'source_Y', 'source_geometry'])
    df_count.to_csv(os.path.join(path_output, "results.csv"))
    # df_count.to_file(os.path.join(path_output, "results.shp"))
    return df_count, dataframe

Check input to the model

In [0]:
def build_edge_list_itertuple(geodataframe, maximum_distance):
  list_dataframe = []
  for row in geodataframe.itertuples():
    print(row.TARGET_FID)
    source_list = [row.TARGET_FID] * len(geodataframe)
    source_list = np.hstack(source_list)
    target_list = list(geodataframe.TARGET_FID)
    frame = { 'Source': source_list, 'Target': target_list } 
    df = pd.DataFrame(frame)

    # calculate distance based on polygon
    polygons = geodataframe[['TARGET_FID', 'geometry']]
    source_poly = polygons[polygons['TARGET_FID'].isin(source_list)]
    source_poly = pd.merge(source_list, polygon, left_on='TARGET_FID', right_on='TARGET_FID', how='left')
    source_poly = gpd.GeoDataFrame(source_poly, geometry='geometry')
    target_poly = polygons[polygons['TARGET_FID'].isin(target_list)]
    target_poly = pd.merge(target_list, polygon, left_on='TARGET_FID', right_on='TARGET_FID', how='left')
    target_poly = gpd.GeoDataFrame(target_poly, geometry='geometry')
    
    distance_series = source_poly.distance(target_poly)
    df['euc_distance'] = distance_series
  

    return

In [107]:
%%time
# set up & load input data
# gdf = load_data("buildings_raw_pts.shp", 1748570, 5426959, 1748841, 5427115)
gdf_polygon = load_data("buildings_raw.shp", 1748412, 5426564, 1749086, 5427606) # smaller
# gdf_polygon = load_data("buildings_raw.shp", 1747550, 5426440, 1748813, 5428346) # comparison
gdf_polygon["area"] = gdf_polygon['geometry'].area  # m2
gdf = gdf_polygon.copy()
gdf['geometry'] = gdf['geometry'].centroid
gdf['X'] = gdf.centroid.x
gdf['Y'] = gdf.centroid.y
gdf['d_short'] = gdf_polygon.exterior.distance(gdf)
gdf['d_long'] = gdf['area'] / gdf['d_short']

CPU times: user 153 ms, sys: 20 ms, total: 173 ms
Wall time: 193 ms


In [145]:
%%time
geodataframe = gdf_polygon
maximum_distance = 45
for row in geodataframe.head(1).itertuples():
  source_list = [row.TARGET_FID] * len(geodataframe)
  source_list = np.hstack(source_list)
  target_list = list(geodataframe.TARGET_FID)
  frame = { 'Source': source_list, 'Target': target_list } 
  df = pd.DataFrame(frame)

  # create edgelist geodataframe to calculate distance and bearing
  polygons = geodataframe[['TARGET_FID', 'geometry']]
  in_source_list = polygons['TARGET_FID'].isin(source_list)

  # create a df from the source row selected of the length source list
  source_df = polygons_crop.append([polygons_crop]*(len(source_list)-1),ignore_index=True)

  # add geometry to dataset
  df['source_geometry'] = source_df['geometry']
  df['target_geometry'] = polygons['geometry']

  # convert to geodataframe and calculate centroid X Y 
  source_geometries = gpd.GeoDataFrame(df, geometry='source_geometry')
  source_geometries['X'] = source_geometries.centroid.x
  source_geometries['Y'] = source_geometries.centroid.y

  target_geometries = gpd.GeoDataFrame(df, geometry='target_geometry')
  target_geometries['X'] = target_geometries.centroid.x
  target_geometries['Y'] = target_geometries.centroid.y

  # add coordinate columns to df
  df['source_x'] = source_geometries['X'] 
  df['target_x'] = target_geometries['X']
  df['source_y'] = source_geometries['Y']
  df['target_y'] = target_geometries['Y']

  # calculate bearing between centroids
  df['v1'] = df['source_x'] - df['target_x']
  df['v2'] = df['source_y'] - df['target_y']
  # v1 = source_geometries['X'] - target_geometries['X']
  # v2 = source_geometries['Y'] - target_geometries['Y']
  df['azimuth'] = np.degrees(np.arctan2(v2, v1))
  df['bearing'] = (azimuth + 360) % 360

  # calculate distance
  distance_series = source_geometries.distance(target_geometries)
  df['distance'] = distance_series

  # filter data based on distance
  not_himself = df['distance'] != 0
  above_the_threshold = df['distance'] < maximum_distance
  df = df[(not_himself) & (above_the_threshold)]
  

CPU times: user 118 ms, sys: 0 ns, total: 118 ms
Wall time: 118 ms


In [146]:
df

,Source,Target,source_geometry,target_geometry,X,Y,source_x,target_x,source_y,target_y,v1,v2,azimuth,bearing,distance
551,228,31228,"POLYGON ((1748535.671 5427153.259, 1748539.491...","POLYGON ((1748553.202 5427184.011, 1748556.342...",1.748562e+06,5.427184e+06,1.748562e+06,1.748562e+06,5.427184e+06,5.427184e+06,0.0,0.0,0.0,0.0,35.234164
552,228,31229,"POLYGON ((1748535.671 5427153.259, 1748539.491...","POLYGON ((1748540.672 5427197.635, 1748552.579...",1.748543e+06,5.427186e+06,1.748543e+06,1.748543e+06,5.427186e+06,5.427186e+06,0.0,0.0,0.0,0.0,22.491217
553,228,31230,"POLYGON ((1748535.671 5427153.259, 1748539.491...","POLYGON ((1748577.361 5427172.863, 1748585.274...",1.748577e+06,5.427161e+06,1.748577e+06,1.748577e+06,5.427161e+06,5.427161e+06,0.0,0.0,0.0,0.0,28.380761
554,228,31231,"POLYGON ((1748535.671 5427153.259, 1748539.491...","POLYGON ((1748569.548 5427178.412, 1748577.155...",1.748568e+06,5.427165e+06,1.748568e+06,1.748568e+06,5.427165e+06,5.427165e+06,0.0,0.0,0.0,0.0,20.250856
555,228,31232,"POLYGON ((1748535.671 5427153.259, 1748539.491...","POLYGON ((1748550.024 5427179.550, 1748565.368...",1.748554e+06,5.427169e+06,1.748554e+06,1.748554e+06,5.427169e+06,5.427169e+06,0.0,0.0,0.0,0.0,13.494665
556,228,31233,"POLYGON ((1748535.671 5427153.259, 1748539.491...","POLYGON ((1748538.571 5427204.338, 1748537.308...",1.748524e+06,5.427176e+06,1.748524e+06,1.748524e+06,5.427176e+06,5.427176e+06,0.0,0.0,0.0,0.0,15.727258
557,228,31234,"POLYGON ((1748535.671 5427153.259, 1748539.491...","POLYGON ((1748504.384 5427143.083, 1748510.298...",1.748529e+06,5.427128e+06,1.748529e+06,1.748529e+06,5.427128e+06,5.427128e+06,0.0,0.0,0.0,0.0,7.519344
558,228,31235,"POLYGON ((1748535.671 5427153.259, 1748539.491...","POLYGON ((1748553.132 5427135.278, 1748557.584...",1.748561e+06,5.427134e+06,1.748561e+06,1.748561e+06,5.427134e+06,5.427134e+06,0.0,0.0,0.0,0.0,17.997931
559,228,31236,"POLYGON ((1748535.671 5427153.259, 1748539.491...","POLYGON ((1748579.566 5427112.333, 1748574.682...",1.748577e+06,5.427125e+06,1.748577e+06,1.748577e+06,5.427125e+06,5.427125e+06,0.0,0.0,0.0,0.0,38.269400
560,228,31237,"POLYGON ((1748535.671 5427153.259, 1748539.491...","POLYGON ((1748542.862 5427094.725, 1748544.264...",1.748545e+06,5.427097e+06,1.748545e+06,1.748545e+06,5.427097e+06,5.427097e+06,0.0,0.0,0.0,0.0,24.039029


In [0]:
%%time
tl0 = datetime.datetime.now()
# create edge list and network
edges = build_edge_list(gdf, 45, gdf_polygon)

# create edges
G = create_network(edges)

tl1 = datetime.datetime.now()
print("creating edge list took : {}".format(tl1 - tl0))